# Making Fruit classifier with VGG16

# Loading VGG16 model

In [1]:
from keras.applications import vgg16

img_rows = 100
img_cols = 100 

model =   vgg16.VGG16(weights = 'imagenet', 
                     include_top = False, 
                     input_shape = (img_rows, img_cols, 3))

# Inspecting each layer

In [2]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


# Freezing all layers except the top 4

In [3]:
for layer in model.layers:
    layer.trainable = False
 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# Making a function that returns our FC head/top

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

# Adding our FC head back onto VGG

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
#from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 131

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

# Loading our fruits dataset

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'fruits-360_dataset/fruits-360/Training'
validation_data_dir = 'fruits-360_dataset/fruits-360/Test'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


# Training our top layers

In [7]:


modelnew.compile(loss = 'categorical_crossentropy',
              optimizer =  "adam"  ,
              metrics = ['accuracy'])

nb_train_samples = 67692
nb_validation_samples = 22688
epochs = 20
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    #callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("tlfinal.h5")

<ipython-input-7-50c24733b27d>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = modelnew.fit_generator(


Epoch 1/20
4230/4230 [==============================] - 7295s 2s/step - loss: 1.5138 - accuracy: 0.5923 - val_loss: 0.6407 - val_accuracy: 0.8396
Epoch 2/20
4230/4230 [==============================] - 5084s 1s/step - loss: 0.6325 - accuracy: 0.8006 - val_loss: 0.4891 - val_accuracy: 0.8839
Epoch 3/20
4230/4230 [==============================] - 5133s 1s/step - loss: 0.5032 - accuracy: 0.8375 - val_loss: 0.3970 - val_accuracy: 0.9073
Epoch 4/20
4230/4230 [==============================] - 6473s 2s/step - loss: 0.4499 - accuracy: 0.8544 - val_loss: 0.3867 - val_accuracy: 0.9130
Epoch 5/20
4230/4230 [==============================] - 5157s 1s/step - loss: 0.4108 - accuracy: 0.8662 - val_loss: 0.4603 - val_accuracy: 0.8947
Epoch 6/20
4230/4230 [==============================] - 5102s 1s/step - loss: 0.3986 - accuracy: 0.8710 - val_loss: 0.4072 - val_accuracy: 0.9140
Epoch 7/20
4230/4230 [==============================] - 5171s 1s/step - loss: 0.3796 - accuracy: 0.8766 - val_loss: 0.4929 -